In [18]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import os as _dir

csv_filelist = ["Mouse_metadata.csv","Study_results.csv"]
working_dir = _dir.getcwd()
for csv in csv_filelist:   
    for root, dirs, files in _dir.walk(working_dir):
        if csv in files:
            if csv == csv_filelist[0]:
                mouse_metadata_path = (_dir.path.join(root, csv))
            else:
               study_results_path =  (_dir.path.join(root, csv))
# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
combine_resultset = pd.merge(study_results, mouse_metadata, on='Mouse ID')
# Display the data table for preview
combine_resultset.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.000000,0,Capomulin,Female,9,22
1,b128,5,45.651331,0,Capomulin,Female,9,22
2,b128,10,43.270852,0,Capomulin,Female,9,22
3,b128,15,43.784893,0,Capomulin,Female,9,22
4,b128,20,42.731552,0,Capomulin,Female,9,22


In [19]:
# Get Overall outlook of DF
combine_resultset.describe()

,Timepoint,Tumor Volume (mm3),Metastatic Sites,Age_months,Weight (g)
count,1893.000000,1893.000000,1893.000000,1893.000000,1893.000000
mean,19.572108,50.448381,1.021659,12.814580,25.662441
std,14.079460,8.894722,1.137974,7.189592,3.921622
min,0.000000,22.050126,0.000000,1.000000,15.000000
25%,5.000000,45.000000,0.000000,7.000000,25.000000
50%,20.000000,48.951474,1.000000,13.000000,27.000000
75%,30.000000,56.292200,2.000000,20.000000,29.000000
max,45.000000,78.567014,4.000000,24.000000,30.000000


In [21]:
 # Checking the number of mice.
combine_resultset['Mouse ID'].describe()
# combine_resultset['Mouse ID'].value_counts().head()

count     1893
unique     249
top       g989
freq        13
Name: Mouse ID, dtype: object

In [16]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint.
# Get all the data for the duplicate mouse ID. 
combine_resultset.loc[combine_resultset.duplicated(subset=['Mouse ID','Timepoint'])== True]

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
861,g989,0,45.000000,0,Propriva,Female,21,26
863,g989,5,47.570392,0,Propriva,Female,21,26
865,g989,10,49.880528,0,Propriva,Female,21,26
867,g989,15,53.442020,0,Propriva,Female,21,26
869,g989,20,54.657650,1,Propriva,Female,21,26


In [17]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
combine_resultset = combine_resultset.set_index('Mouse ID').drop(labels='g989').reset_index()

g791    10
i901    10
s710    10
q511    10
f545    10
Name: Mouse ID, dtype: int64

In [ ]:
# Checking the number of mice in the clean DataFrame.
combine_resultset['Mouse ID'].value_counts().head()